In [1]:
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib

from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
GeoAxes._pcolormesh_patched = Axes.pcolormesh
from scipy.interpolate import interpn

#from mpl_toolkits.basemap import Basemap
import math
import warnings
%matplotlib inline
import xarray as xr
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', message='All-NaN slice encountered')

from matplotlib import cm
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

In [3]:
# %load latlon_functions.py
def interpolate(xarr,yarr,ds):
    """
    inputs:
        ds: the dataset with lat and long values
        xarr: the current row of the Xgrid array
        yarr: the current row of the Ygrid array
    returns:
        points: the lat/lon values of the current array
    """
    lat_values = ds.lat_rho.values
    long_values = ds.lon_rho.values
    lat = np.array([])
    long = np.array([])
    
    for i,j in zip(xarr,yarr):
        # interpolate in latitude
        x_shape = np.arange(ds.lat_rho.shape[0])
        y_shape = np.arange(ds.lat_rho.shape[1])
        interp_x = i
        interp_y = j
        # Note the following two lines that are used to set up the
        interp_mesh = np.array(np.meshgrid(interp_x, interp_y))
        interp_points = np.rollaxis(interp_mesh, 0, 3).reshape((1, 2))
        # Perform the interpolation
        interp_arr = interpn((x_shape, y_shape), lat_values, interp_points)
        lat = np.append(interp_arr[0],lat)
        
        # interpolate in longitude
        x_shape = np.arange(ds.lon_rho.shape[0])
        y_shape = np.arange(ds.lon_rho.shape[1])
        interp_x = j
        interp_y = i
        # Note the following two lines that are used to set up the
        interp_mesh = np.array(np.meshgrid(interp_x, interp_y))
        interp_points = np.rollaxis(interp_mesh, 0, 3).reshape((1, 2))
        # Perform the interpolation
        interp_arr = interpn((x_shape, y_shape), long_values, interp_points)
        long = np.append(interp_arr[0],long)
    return (long,lat)


    # write a function to get the long and lat from the grid points
def grid_to_spherical(x,y,ds):
    """
    x: array of grid x position values
    y: array of grid y position values
    rho: the used xi_rho value 
    """
    x_long = np.empty((0,x.shape[1]))
    y_lat = np.empty((0,y.shape[1]))
    
    # first create the points in the other array
    # get the maximum and minimum values of x and y
    for i in range(len(x)):
        
        # get the min and max of x and y
        minx = (np.nanmin(x[i]))
        maxx = (np.nanmax(x[i]))
        if not (math.isnan(minx)) and not (math.isnan(maxx)):
            x_i,y_i = interpolate(x[i],y[i],ds)
        else:
            x_i,y_i = x[i],y[i]
        x_long = np.append(np.array([x_i]),x_long,axis=0)
        y_lat = np.append(np.array([y_i]),y_lat,axis=0)
        
    return (x_long,y_lat)


In [4]:
ds = xr.open_dataset('/scratch/project_2000789/muramarg/run_5_27/output_WAOM_check/ocean_flt.nc')
dg = xr.open_dataset('/scratch/project_2000789/boeiradi/waom10_frc/waom10extend_grd.nc')

In [ ]:
# create the latitude and longitude points from the spherical grids
x = (ds.variables['Xgrid'].values)
y = (ds.variables['Ygrid'].values)
long,lat = grid_to_spherical(x,y,dg)

In [ ]:
# write the new latitude and longitude points to a file
lines = []
for i in (long):
    line = ""
    for x in i:
        mystr = str(x)
        line += mystr
        line += ","
    # create the line to add to lines
    lines.append(line)
with open('lonpoints.txt', 'w') as f:
    for line in lines:
        f.write(line)
        f.write('\n')
f.close()

lines=[]
for i in (lat):
    line = ""
    for x in i:
        mystr = str(x)
        line += mystr
        line += ","
    # create the line to add to lines
    lines.append(line)
with open('latpoints.txt', 'w') as f:
    for line in lines:
        f.write(line)
        f.write('\n')
f.close()